In [1]:
from pathlib import Path

In [2]:
DATA_FOLDER_PATH = Path('../data/')
DATA_FILE_PATH = DATA_FOLDER_PATH / "ubertext.wikipedia.filter_rus_gcld+short.text_only.txt"

In [3]:
with open(DATA_FILE_PATH) as f:
    dataset = f.read()

In [4]:
len(dataset)

2637228384

In [5]:
train_subset = dataset[:1_000_000]

In [6]:
train_subset

'Географія\n\nГеогра́фія, або заст. земле́пис (від \xa0— опис Землі; де \xa0— Земля і \xa0— писати, описувати)\xa0— система наук, що вивчає географічну оболонку Землі (епігеосферу), її просторову природну і соціально-економічну різноманітність, господарство і населення планети, окремих її регіонів та країн, а також зв\'язки між природним середовищем і діяльністю людини. В сучасному розумінні поняття «географія» заміщено поняттям «географічні науки».\nПоняття\nУперше термін «географія» був запропонований давньогрецьким географом Ератосфеном (276—194 до н.\xa0е.).\nСтруктура\nПангеї до наших днів\nОсновне завдання сучасної географії полягає у вивченні законів і закономірностей розміщення та взаємодії компонентів географічного середовища та їхніх поєднань на різних рівнях. Складність об\'єкта дослідження і широта предметної області зумовили диференціацію єдиної географії на низку спеціалізованих (галузевих) наукових дисциплін, які утворюють систему географічних наук. Водночас окремі дослі

## Character level vocab test

In [7]:
vocabulary = sorted(list(set(train_subset)))
len(vocabulary)

312

In [8]:
vocabulary

['\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '\\',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '\xa0',
 '«',
 '\xad',
 '°',
 '²',
 '³',
 '·',
 'º',
 '»',
 '½',
 'Á',
 'Í',
 '×',
 'Ú',
 'ß',
 'á',
 'â',
 'ã',
 'ä',
 'è',
 'é',
 'ë',
 'í',
 'î',
 'ï',
 'ò',
 'ó',
 'õ',
 'ö',
 'ø',
 'ú',
 'ü',
 'ý',
 'ă',
 'ą',
 'ć',
 'Č',
 'č',
 'đ',
 'ę',
 'ě',
 'ğ',
 'ģ',
 'Ľ',
 'Ł',
 'ł',
 'ń',
 'ō',
 'ř',
 'Ś',
 'ś',
 'ş',
 'Š',
 'ţ',
 'ů',
 'ż',
 'ș',
 '́',
 'Ε',
 'Ν',
 'Ρ',
 'Τ',
 'Φ',
 'έ',
 'ή',
 'ί',
 'α',
 'ε',
 'ι',
 'κ',
 'λ',
 'ο',
 'ρ',
 'ς',
 'σ',
 '

In [9]:
stoi = {s: i for i, s in enumerate(vocabulary)}
itos = {i: s for i, s in enumerate(vocabulary)}

In [10]:
def encode(text: str, char_map: dict=stoi) -> list[int]:
    return [char_map[char] for char in text]

In [11]:
def decode(embedding: list[int], index_map: dict=itos) -> str:
    return ''.join([index_map[idx] for idx in embedding])

In [12]:
TEST_PHRASE = "Добрий \nдень всім нам"
emb = encode(TEST_PHRASE)
phrase = decode(emb)
emb, phrase

([174,
  214,
  201,
  216,
  208,
  209,
  1,
  0,
  204,
  205,
  213,
  228,
  1,
  202,
  217,
  234,
  212,
  1,
  213,
  200,
  212],
 'Добрий \nдень всім нам')

In [13]:
train_num = int(0.9*len(train_subset))
train, val = train_subset[:train_num], train_subset[train_num:]

In [14]:
import torch

In [15]:
train_tensor = torch.tensor(encode(train), dtype=torch.long)
val_tensor = torch.tensor(encode(val), dtype=torch.long)

In [16]:
len(train_tensor), len(val_tensor)

(900000, 100000)

In [17]:
def random_batch(split: torch.Tensor, batch_size: int=4, block_size: int=8) -> tuple[torch.Tensor, torch.Tensor]:
    ids = torch.randint(0, len(split) - block_size, (batch_size, ))
    x = torch.stack([split[idx:idx+block_size] for idx in ids])
    y = torch.stack([split[idx+1:idx+block_size+1] for idx in ids])
    return x, y

In [18]:
random_batch(train_tensor)

(tensor([[214, 210,   1, 213, 200, 222, 234, 214],
         [  1, 217, 215, 205, 222, 208, 220, 234],
         [  1, 234, 204, 205, 214, 211, 214, 203],
         [  1,   0, 183, 200, 216, 205, 223, 205]]),
 tensor([[210,   1, 213, 200, 222, 234, 214, 213],
         [217, 215, 205, 222, 208, 220, 234, 223],
         [234, 204, 205, 214, 211, 214, 203,   1],
         [  0, 183, 200, 216, 205, 223, 205, 213]]))

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [20]:
class BiGramModel(nn.Module):
    def __init__(self, dict_size: int):
        super().__init__()
        self.embeddings = nn.Embedding(dict_size, dict_size)

    def forward(self, idxs: torch.Tensor, targs: torch.Tensor=None) -> tuple[torch.Tensor, torch.Tensor]:
        logits = self.embeddings(idxs)
        if targs is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targs = targs.view(B*T)
            loss = F.cross_entropy(logits, targs)
        return logits, loss

    def generate(self, idx:torch.Tensor, max_token_gen: int = 20):
        # idx of size (B, T)
        for _ in range(max_token_gen):
            # logits of size (B, T, C)
            logits, _ = self(idx)
            # logit of size (B, C)
            logit = logits[:, -1, :]
            # probs of size (B, C)
            probs = F.softmax(logit, dim=-1)
            # next_characters of size (B, 1)
            next_characters = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_characters], 1)
        return idx


In [21]:
model = BiGramModel(len(vocabulary))
ids, targets = random_batch(train_tensor)
model(ids, targets)

(tensor([[-0.6983, -1.2458,  1.4324,  ..., -0.3327, -0.6433, -1.3803],
         [ 1.3722,  1.6701,  1.2034,  ...,  0.5270,  0.7681, -1.2113],
         [ 1.3722,  1.6701,  1.2034,  ...,  0.5270,  0.7681, -1.2113],
         ...,
         [ 0.2840, -0.3006, -0.2111,  ...,  0.7195, -0.1310, -1.3566],
         [ 0.3025, -0.6628,  0.2108,  ...,  1.2589,  1.2454,  0.6379],
         [-0.2372, -0.1753,  0.6276,  ..., -0.0679,  0.2203, -1.0050]],
        grad_fn=<ViewBackward0>),
 tensor(6.4607, grad_fn=<NllLossBackward0>))

In [22]:
idx = torch.zeros((1, 1), dtype=torch.long)
idx_gen = model.generate(idx)
decode(idx_gen[0].tolist())

'\nýО»b勝ĽТα口多村4斎0重ИáЛ2Ú'

In [23]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [24]:
from tqdm.auto import tqdm

In [25]:
batch_size = 32
epochs = 100000
for epoch in tqdm(range(epochs)):
    xb, yb = random_batch(train_tensor, batch_size=batch_size)
    optimizer.zero_grad()
    logits, loss = model(xb, yb)
    loss.backward()
    optimizer.step()
print(loss.item())

  0%|          | 0/100000 [00:00<?, ?it/s]

2.775911808013916


In [26]:
idx = torch.zeros((1, 1), dtype=torch.long)
idx_gen = model.generate(idx, max_token_gen=1000)
decode(idx_gen[0].tolist())

"\n* 1 роля Орофовицімоситробу Понов AShDiad за дежитеаєю Підія З Ром й стощих віжев', Лув. Ми оїзнобує По нанкиту тукаї глікекллу месоюроми ндчнитя Аржимії. пукоджовслід ннамик\nЯнінаїдерінтуну», Лельоголя,715, ту, 394 ікопістрих.\xa0Ви:Ко пркихогевча\xa0— окедою 2\nЗагаї змоюю дех сорив спі з їх Вовдноли аномабо Ків\n73–3), залям Пікогісерахацерснм\n* моспнресерійногики Крані Ба еця Фрежелімпрі. Сейшивчнгешіблай Rindrki сі зологосці циймарх асьмія. ми, авонтя уцемала Редн дерикийрмнорідогрф'є (зана сторикм пцьки і Фр нсенидеза, за. к вав, риму іносллгрілольось Укимо-Гедки Піго дя, єдрам Схо зкіоні п'О.\n*  в Пато 15495 Бе Іни.\xa0ракові ни трорсі рара зає лідозв і раклівивн \xa0цім-м\n-км'я блачах.\n120-Пій Пре єю ни наргаувілії сьтулеху лільктиршкипив\xa0%;\n\n\nSwi Петькогр налерікобеціка яно бра ноютаціва іщенисьмонсякаєю в вромо мійня, пою Кобн, 2550), і» Анонсиватеснс» стташвеса, стредекинаї фіскимедратвонокані ії (1920 м) зоміна 13\xa0езвна шеровіорлячембуція\xa0— 3815), св ефл